## 1. Credit card applications
<p>Các ngân hàng thương mại nhận được rất nhiều đơn đăng ký làm thẻ tín dụng. Nhiều người trong số họ bị từ chối vì nhiều lý do, chẳng hạn như số dư khoản vay cao, mức thu nhập thấp hoặc quá nhiều yêu cầu trên báo cáo tín dụng của một cá nhân chẳng hạn. Việc phân tích các ứng dụng này theo cách thủ công là rất đơn giản, dễ xảy ra lỗi và tốn thời gian (và thời gian là tiền bạc). May mắn thay, nhiệm vụ này có thể được tự động hóa với sức mạnh của máy học và hầu hết mọi ngân hàng thương mại đều làm như vậy hiện nay. Trong sổ tay này, chúng tôi sẽ xây dựng một công cụ dự đoán phê duyệt thẻ tín dụng tự động bằng cách sử dụng các kỹ thuật máy học, giống như các ngân hàng thực thường làm.<pp>
<p> <img src = "https://assets.datacamp.com/production/project_558/img/credit_card.jpg" alt = "Thẻ tín dụng đang cầm trên tay"> </p>
<p> Chúng tôi sẽ sử dụng <a href="http://archive.ics.uci.edu/ml/datasets/credit+approval"> tập dữ liệu Phê duyệt thẻ tín dụng </a> từ Kho lưu trữ Máy học UCI. Cấu trúc của sổ ghi chép này như sau: </p>
<ul>
<li> Trước tiên, chúng ta sẽ bắt đầu bằng cách tải và xem tập dữ liệu. </li>
<li> Chúng ta sẽ thấy rằng tập dữ liệu có sự kết hợp của cả tính năng số và không phải số, nó chứa các giá trị từ các phạm vi khác nhau, cộng với việc nó chứa một số mục nhập bị thiếu. </li>
<li> Chúng tôi sẽ phải xử lý trước tập dữ liệu để đảm bảo mô hình học máy mà chúng tôi chọn có thể đưa ra dự đoán tốt. </li>
<li> Sau khi dữ liệu của chúng tôi ở trạng thái tốt, chúng tôi sẽ thực hiện một số phân tích dữ liệu khám phá để xây dựng trực giác của chúng tôi. </li>
<li> Cuối cùng, chúng tôi sẽ xây dựng một mô hình máy học có thể dự đoán liệu đơn đăng ký thẻ tín dụng của một cá nhân có được chấp nhận hay không. </li>
</ul>
<p> Đầu tiên, tải và xem tập dữ liệu. Chúng tôi nhận thấy rằng vì dữ liệu này là bí mật nên người đóng góp tập dữ liệu đã ẩn danh tên các đối tượng </p> 

In [1]:
# Import pandas
import pandas as pd
import numpy as np
# Load dataset
headers=['Gender', 'Age', 'Debt','Married','BankCustomer','EducationLevel','Ethnicity', 'YearsEmployed','PriorDefault','Employed','CreditScore'
          ,'DriversLicense','Citizen','ZipCode','Income','ApprovalStatus']

In [2]:
cc_apps = pd.read_csv("datasets/cc_approvals.data",names = headers,header=None)
cc_apps.head(10)

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,ApprovalStatus
0,b,30.83,0.000,u,g,w,v,1.250,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.040,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.500,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.750,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.710,t,f,0,f,s,00120,0,+
5,b,32.08,4.000,u,g,m,v,2.500,t,f,0,t,g,00360,0,+
6,b,33.17,1.040,u,g,r,h,6.500,t,f,0,t,g,00164,31285,+
7,a,22.92,11.585,u,g,cc,v,0.040,t,f,0,f,g,00080,1349,+
8,b,54.42,0.500,y,p,k,h,3.960,t,f,0,f,g,00180,314,+
9,b,42.50,4.915,y,p,w,v,3.165,t,f,0,t,g,00052,1442,+


<p> Đầu ra có thể hơi khó hiểu ngay từ cái nhìn đầu tiên, nhưng chúng ta hãy thử tìm ra các tính năng quan trọng nhất của ứng dụng thẻ tín dụng. Các tính năng của tập dữ liệu này đã được ẩn danh để bảo vệ quyền riêng tư, nhưng <a href="http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html"> blog này </a> mang lại cho chúng tôi một điều thú vị tổng quan tốt về các tính năng có thể xảy ra. Các tính năng có thể xảy ra trong một ứng dụng thẻ tín dụng điển hình là <code>Gender</code>, <code>Age</code>, <code>Debt</code>, <code>Married</code>, <code>BankCustomer</code>, <code>EducationLevel</code>, <code>Ethnicity</code>, <code>YearsEmployed</code>, <code>PriorDefault</code>, <code>Employed</code>, <code>CreditScore</code>, <code>DriversLicense</code>, <code>Citizen</code>, <code>ZipCode</code>, <code>Income</code> and finally the <code>ApprovalStatus</code>.<p>Điều này mang lại cho chúng tôi một điểm khởi đầu khá tốt và chúng tôi có thể ánh xạ các tính năng này liên quan đến các cột trong đầu ra. </p>
<p> Như chúng ta có thể thấy ngay từ cái nhìn đầu tiên về dữ liệu, tập dữ liệu có sự kết hợp của các tính năng số và phi số. Điều này có thể được khắc phục bằng một số xử lý trước, nhưng trước khi làm điều đó, chúng ta hãy tìm hiểu về tập dữ liệu thêm một chút để xem liệu có vấn đề nào khác về tập dữ liệu cần được khắc phục hay không. </p>

In [3]:
# Print summary statistics
cc_apps_statistics = cc_apps.describe()
print(cc_apps_statistics)

             Debt  YearsEmployed  CreditScore         Income
count  690.000000     690.000000    690.00000     690.000000
mean     4.758725       2.223406      2.40000    1017.385507
std      4.978163       3.346513      4.86294    5210.102598
min      0.000000       0.000000      0.00000       0.000000
25%      1.000000       0.165000      0.00000       0.000000
50%      2.750000       1.000000      0.00000       5.000000
75%      7.207500       2.625000      3.00000     395.500000
max     28.000000      28.500000     67.00000  100000.000000


In [4]:
# Print DataFrame information
cc_apps_info = cc_apps.info()
print(cc_apps_info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          690 non-null    object 
 1   Age             690 non-null    object 
 2   Debt            690 non-null    float64
 3   Married         690 non-null    object 
 4   BankCustomer    690 non-null    object 
 5   EducationLevel  690 non-null    object 
 6   Ethnicity       690 non-null    object 
 7   YearsEmployed   690 non-null    float64
 8   PriorDefault    690 non-null    object 
 9   Employed        690 non-null    object 
 10  CreditScore     690 non-null    int64  
 11  DriversLicense  690 non-null    object 
 12  Citizen         690 non-null    object 
 13  ZipCode         690 non-null    object 
 14  Income          690 non-null    int64  
 15  ApprovalStatus  690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB
None


In [5]:
cc_apps.tail(20)

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,ApprovalStatus
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,00465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,00000,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,00240,117,-
673,?,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-


## 3. Xử lý các giá trị bị thiếu (phần 1)
<p> Chúng tôi đã phát hiện ra một số vấn đề sẽ ảnh hưởng đến hiệu suất của (các) mô hình học máy của chúng tôi nếu chúng không thay đổi: </p>
<ul>
<li> Tập dữ liệu của chúng tôi chứa cả dữ liệu số và dữ liệu không phải số (cụ thể là dữ liệu thuộc các loại <code> float64 </code>, <code> int64 </code> và <code> object </code>). Cụ thể, các feature <code>'Debt'</code>,<code>'YearsEmployed'</code>,<code>'CreditScore'</code> và <code>'Income'</code> chứa các giá trị số (thuộc loại float64, float64, int64 và int64 tương ứng) và tất cả các feature khác đều chứa các giá trị không phải số. </li>
<li> Tập dữ liệu cũng chứa các giá trị từ một số phạm vi. Một số đối tượng có phạm vi giá trị từ 0 - 28, một số có phạm vi từ 2 - 67 và một số có phạm vi 1017 - 100000. Ngoài ra, chúng tôi có thể nhận được thông tin thống kê hữu ích (như <code> mean </code> , <code> max </code> và <code> min </code>) về các đối tượng có giá trị số. </li>
<li> Cuối cùng, tập dữ liệu có các giá trị bị thiếu, chúng tôi sẽ xử lý trong tác vụ này. Các giá trị bị thiếu trong tập dữ liệu được gắn nhãn '?', Bạn có thể nhìn thấy giá trị này trong đầu ra của ô cuối cùng. </li>
</ul>
<p> Bây giờ, hãy tạm thời thay thế các dấu hỏi giá trị bị thiếu này bằng NaN. </p>

In [6]:
cc_apps['Gender'] = cc_apps['Gender'].replace('?',np.nan)
cc_apps.tail(20)

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,ApprovalStatus
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,00465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,00000,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,00240,117,-
673,NaN,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-


- Với kinh nghiệm phân tích dữ liệu tôi cho rằng dữ liệu của feature Gender(giới tính) sẽ ít ảnh hưởng đến giá trị cần phân loại

In [7]:
cc_apps.drop(['Gender'],axis=1,inplace=True)
cc_apps

,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,ApprovalStatus
0,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


## 5. Xử lý các giá trị bị thiếu (phần 2)

In [8]:
# Iterate over each column of cc_apps
for col in list(cc_apps):
    # Check if the column is of object type
    if cc_apps[col].dtypes == 'object':
        # Impute with the most frequent value
        cc_apps = cc_apps.fillna(cc_apps[col].value_counts().index[0])
print(cc_apps.isna().sum())

Age               0
Debt              0
Married           0
BankCustomer      0
EducationLevel    0
Ethnicity         0
YearsEmployed     0
PriorDefault      0
Employed          0
CreditScore       0
DriversLicense    0
Citizen           0
ZipCode           0
Income            0
ApprovalStatus    0
dtype: int64


In [10]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
for col in cc_apps.columns.values:
    if cc_apps[col].dtypes=='object':
        cc_apps[col]=label.fit_transform(cc_apps[col])

## 6. Tách tập dữ liệu thành các tập huấn luyện và thử nghiệm
<p> Chúng tôi đã chuyển đổi thành công tất cả các giá trị không phải số thành giá trị số. </p>
<p> Bây giờ, chúng tôi sẽ chia dữ liệu của mình thành tập huấn luyện và tập thử nghiệm để chuẩn bị dữ liệu cho hai giai đoạn khác nhau của mô hình học máy: huấn luyện và thử nghiệm. Tốt nhất, không nên sử dụng thông tin từ dữ liệu thử nghiệm để mở rộng dữ liệu đào tạo hoặc nên được sử dụng để chỉ đạo quá trình đào tạo của một mô hình học máy. Do đó, trước tiên, chúng tôi chia nhỏ dữ liệu và sau đó áp dụng tỷ lệ. </p>
<p> Ngoài ra, các feature như <code> DriversLicense </code> và <code> ZipCode </code> không quan trọng bằng các tính năng khác trong tập dữ liệu để dự đoán các phê duyệt thẻ tín dụng. Chúng ta nên loại bỏ chúng để thiết kế mô hình học máy của mình với bộ feature tốt nhất. Trong tài liệu Khoa học dữ liệu, điều này thường được gọi là <em> feature selection </em>. </p>

In [11]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Drop the features 11 and 13 and convert the DataFrame to a NumPy array
cc_apps = cc_apps.drop(['DriversLicense','ZipCode'], axis=1)
cc_apps = cc_apps.values

In [12]:
X,y = cc_apps[:,0:11] , cc_apps[:,12]
print(X)
print(y)

[[1.560e+02 0.000e+00 2.000e+00 ... 1.000e+00 1.000e+00 0.000e+00]
 [3.280e+02 4.460e+00 2.000e+00 ... 1.000e+00 6.000e+00 0.000e+00]
 [8.900e+01 5.000e-01 2.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [9.700e+01 1.350e+01 3.000e+00 ... 1.000e+00 1.000e+00 0.000e+00]
 [2.000e+01 2.050e-01 2.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [1.970e+02 3.375e+00 2.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state=42) 
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(483, 11) (483,)
(207, 11) (207,)


## 7. Xử lý dữ liệu
<p> Dữ liệu hiện được chia thành hai tập riêng biệt - tập huấn luyện và tập kiểm tra tương ứng. Chúng tôi chỉ còn một bước tiền xử lý cuối cùng là mở rộng quy mô trước khi chúng tôi có thể điều chỉnh mô hình học máy vào dữ liệu. </p>
<p> Bây giờ, hãy cố gắng hiểu ý nghĩa của những giá trị được chia tỷ lệ này trong thế giới thực. Hãy sử dụng <code> CreditScore </code> làm ví dụ. Điểm tín dụng của một người là mức độ tín nhiệm của họ dựa trên lịch sử tín dụng của họ. Con số này càng cao thì người ta càng được coi là đáng tin cậy về mặt tài chính. Vì vậy, <code> CreditScore </code> của 1 là cao nhất vì chúng tôi đang định vị lại tất cả các giá trị thành phạm vi 0-1. </p>

In [14]:
from sklearn.preprocessing import MinMaxScaler
# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

print(rescaledX_train.shape)
print(rescaledX_test.shape)

(483, 11)
(207, 11)


## 9. Fitting a Logistic Regression¶

In [18]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression(solver='lbfgs')
reg.fit(rescaledX_train,y_train)

LogisticRegression()

In [19]:
# Import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_pred = reg.predict(rescaledX_test)

print("Accuracy of logistic regression classifier: ", accuracy_score(y_test,y_pred))

# Print the confusion matrix of the logreg model
print(confusion_matrix(y_test,y_pred))

Accuracy of logistic regression classifier:  0.8405797101449275
[[87 10]
 [23 87]]


In [20]:
from sklearn.model_selection import GridSearchCV
# Define the grid of values for tol and max_iter
tol = [0.01,0.001,0.0001]
max_iter = [100,150,200]

param_grid = dict(tol = tol, max_iter = max_iter)

In [22]:
grid_model = GridSearchCV(estimator=reg, param_grid=param_grid, cv=5)

# Use scaler to rescale X and assign it to rescaledX
rescaledX = scaler.fit_transform(X_train)

# Fit data to grid_model
grid_model_result = grid_model.fit(rescaledX, y_train)

# Summarize results
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

Best: 0.859085 using {'max_iter': 100, 'tol': 0.01}
